# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
citydatadf = pd.read_csv("city_data.csv")
citydatadf["Max Temp"] = citydatadf["Max Temp"]-273.15
citydatadf

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Luena,-11.7833,19.9167,26.34,18,7,2.52,AO,1657451091
1,Hong Kong,22.2855,114.1577,32.20,75,32,4.47,HK,1657451091
2,Syriam,16.7722,96.2378,27.06,94,100,3.09,MM,1657450922
3,Lüderitz,-26.6481,15.1594,15.47,71,23,3.00,NaN,1657451091
4,San Quintín,30.4833,-115.9500,18.72,90,15,4.47,MX,1657451091
...,...,...,...,...,...,...,...,...,...
576,Acaraú,-2.8856,-40.1200,25.69,72,85,6.09,BR,1657451151
577,Ust'-Donetskiy,47.6419,40.8718,36.45,15,100,8.81,RU,1657451151
578,Joshīmath,30.5667,79.5667,23.01,89,100,0.90,IN,1657451151
579,Magadan,59.5667,150.8000,14.94,78,2,2.54,RU,1657451151


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
fig = gmaps.figure(map_type="HYBRID")


# Create heat layer
heat_layer = gmaps.heatmap_layer(citydatadf[["Lat","Lng"]], weights=citydatadf["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [116]:


perfecttemp= citydatadf.loc[(citydatadf["Max Temp"]<30) & (citydatadf["Max Temp"]>25),"Max Temp"]
#df2=df.loc[(df['Discount'] >= 1000) & (df['Discount'] <= 2000)]
perfectwind = citydatadf.loc[citydatadf["Wind Speed"]<3.0,"Wind Speed"]
perfectcloud = citydatadf.loc[citydatadf["Cloudiness"]==0,"Cloudiness"]

d = {"Max Temp":perfecttemp,"Wind Speed":perfectwind,"Cloudiness":perfectcloud}
newdf = pd.DataFrame(d)
newdf = newdf.dropna()
newdf = pd.merge(newdf,citydatadf,on=["Max Temp","Wind Speed","Cloudiness"])
newdf

,Max Temp,Wind Speed,Cloudiness,City,Lat,Lng,Humidity,Country,Date
0,25.31,2.06,0.0,Puerto Suárez,-18.9500,-57.8000,83,BO,1657451104
1,27.72,2.18,0.0,Richards Bay,-28.7830,32.0377,26,ZA,1657450869
2,25.25,2.92,0.0,Nokaneng,-19.6667,22.2667,23,BW,1657451107
3,27.82,2.37,0.0,Lata,40.1629,-8.3327,16,PT,1657451112
4,25.91,2.23,0.0,Mocuba,-16.8392,36.9856,48,MZ,1657451114
5,28.33,2.57,0.0,Saint George,37.1041,-113.5841,25,US,1657451038
6,25.86,2.26,0.0,Rundu,-17.9333,19.7667,19,NaN,1657451134
7,28.82,2.45,0.0,Médéa,36.2642,2.7539,21,DZ,1657450991
8,26.67,2.06,0.0,Aurillac,44.9167,2.4500,47,FR,1657451146
9,29.96,0.66,0.0,Mizdah,31.4451,12.9801,24,LY,1657451148


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [117]:
hotel_df = newdf
hotel_df["Hotel Name"] = ""
hotel_df

,Max Temp,Wind Speed,Cloudiness,City,Lat,Lng,Humidity,Country,Date,Hotel Name
0,25.31,2.06,0.0,Puerto Suárez,-18.9500,-57.8000,83,BO,1657451104,
1,27.72,2.18,0.0,Richards Bay,-28.7830,32.0377,26,ZA,1657450869,
2,25.25,2.92,0.0,Nokaneng,-19.6667,22.2667,23,BW,1657451107,
3,27.82,2.37,0.0,Lata,40.1629,-8.3327,16,PT,1657451112,
4,25.91,2.23,0.0,Mocuba,-16.8392,36.9856,48,MZ,1657451114,
5,28.33,2.57,0.0,Saint George,37.1041,-113.5841,25,US,1657451038,
6,25.86,2.26,0.0,Rundu,-17.9333,19.7667,19,NaN,1657451134,
7,28.82,2.45,0.0,Médéa,36.2642,2.7539,21,DZ,1657450991,
8,26.67,2.06,0.0,Aurillac,44.9167,2.4500,47,FR,1657451146,
9,29.96,0.66,0.0,Mizdah,31.4451,12.9801,24,LY,1657451148,


In [147]:
target_radius = 5000
target_type = "Hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotel_df.iterrows():
    target_lat = hotel_df["Lat"][index]
    target_lng = hotel_df["Lng"][index]
    target_coord = str(target_lat) + "," + str(target_lng)
    params = {
    "location": target_coord,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}
    print(f"Retriving a request for Index {index} for lat:{target_lat} and lng: {target_lng}")
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df["Hotel Name"][index] = response["results"][0]["name"]
    except:
        print("Whoopsy, something is wrong here")
        
        

Retriving a request for Index 0 for lat:-18.95 and lng: -57.8


<ipython-input-147-d9dcf3379649>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"][index] = response["results"][0]["name"]


Retriving a request for Index 1 for lat:-28.783 and lng: 32.0377
Retriving a request for Index 2 for lat:-19.6667 and lng: 22.2667
Whoopsy, something is wrong here
Retriving a request for Index 3 for lat:40.1629 and lng: -8.3327
Retriving a request for Index 4 for lat:-16.8392 and lng: 36.9856
Retriving a request for Index 5 for lat:37.1041 and lng: -113.5841
Retriving a request for Index 6 for lat:-17.9333 and lng: 19.7667
Retriving a request for Index 7 for lat:36.2642 and lng: 2.7539
Retriving a request for Index 8 for lat:44.9167 and lng: 2.45
Retriving a request for Index 9 for lat:31.4451 and lng: 12.9801


In [148]:
# results = response["results"]
# results
hotel_df


,Max Temp,Wind Speed,Cloudiness,City,Lat,Lng,Humidity,Country,Date,Hotel Name
0,25.31,2.06,0.0,Puerto Suárez,-18.9500,-57.8000,83,BO,1657451104,Puerto Suárez
1,27.72,2.18,0.0,Richards Bay,-28.7830,32.0377,26,ZA,1657450869,Richards Bay
2,25.25,2.92,0.0,Nokaneng,-19.6667,22.2667,23,BW,1657451107,Mizdah
3,27.82,2.37,0.0,Lata,40.1629,-8.3327,16,PT,1657451112,Coimbra
4,25.91,2.23,0.0,Mocuba,-16.8392,36.9856,48,MZ,1657451114,Mocuba
5,28.33,2.57,0.0,Saint George,37.1041,-113.5841,25,US,1657451038,St. George
6,25.86,2.26,0.0,Rundu,-17.9333,19.7667,19,NaN,1657451134,Rundu
7,28.82,2.45,0.0,Médéa,36.2642,2.7539,21,DZ,1657450991,Médéa
8,26.67,2.06,0.0,Aurillac,44.9167,2.4500,47,FR,1657451146,Aurillac
9,29.96,0.66,0.0,Mizdah,31.4451,12.9801,24,LY,1657451148,Mizdah


In [137]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [143]:
# for index,row in hotel_df.iterrows():
#     target_lat = hotel_df["Lat"][index]
#     target_lng = hotel_df["Lng"][index]
#     target_coord = str(target_lat) + "," + str(target_lng)
    
#     markers = gmaps.marker_layer(target_coord)
#     try:
# # Add the layer to the map
#         fig.add_layer(markers)
#     except:
#         print("whoops")
# fig

ValueError: not enough values to unpack (expected 2, got 1)

In [144]:
 # Assign the marker layer to a variable
markers = gmaps.marker_layer(target_coord)
# Add the layer to the map
fig.add_layer(markers)
fig

ValueError: not enough values to unpack (expected 2, got 1)